# Search Directory Tutorial

This notebook demonstrates how to use the `SearchDirectory` class to create embeddings and query a set of documents using a FAISS index.

In [ ]:
from file_processing import SearchDirectory